In [1]:
seed = 0
import torch
torch.manual_seed(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
# torch.use_deterministic_algorithms(True)

In [2]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
from utils import train, test, edgeindex2adj
# citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
# pubmed = Planetoid(root='.', name='Pubmed')

In [3]:
from models import ALP, GCN
dataset = cora
yshape = dataset[0].y.shape[0]
model = ALP(num_layers=16, alpha=0.9, yshape=yshape)
outs = model.lpa(dataset)

gcn = GCN(dataset.num_features, 8, dataset.num_classes)

optimizer_gcn = torch.optim.Adam(gcn.parameters(), lr=0.005, weight_decay=5e-4)
optimizer_alp = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [4]:
data = dataset[0]
data.train_mask[torch.randperm(data.train_mask.shape[0])[:int(0.9 * data.train_mask.shape[0])]] = False

best_val_acc = final_test_acc = 0
for epoch in range(1, 500):
    
    optimizer_gcn.zero_grad()
    optimizer_alp.zero_grad()
    
    train_mask = data.train_mask.clone()
    val_mask = data.val_mask.clone()
    
    # put 90% of it to 0
    val_mask[torch.randperm(val_mask.shape[0])[:int(0.5 * val_mask.shape[0])]] = False
    train_mask[data.val_mask] = 1
        
    op_alp = model()[train_mask].argmax(dim=-1)
    op = gcn(data.x, data.edge_index)
    
    print(op_alp.shape, op.shape)
    loss = F.cross_entropy(op[train_mask, :], op_alp)
    loss.backward()
    
    optimizer_alp.step()
    optimizer_gcn.step()
    
    train_acc, val_acc, tmp_test_acc = test(gcn, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

torch.Size([509]) torch.Size([2708, 7])
Epoch: 001, Loss: 1.9463244676589966, Train: 0.2222, Val: 0.3360, Test: 0.3130
torch.Size([509]) torch.Size([2708, 7])
Epoch: 002, Loss: 1.9089198112487793, Train: 0.3333, Val: 0.3700, Test: 0.3520
torch.Size([509]) torch.Size([2708, 7])
Epoch: 003, Loss: 1.8704973459243774, Train: 0.2222, Val: 0.3820, Test: 0.3630
torch.Size([509]) torch.Size([2708, 7])
Epoch: 004, Loss: 1.830008864402771, Train: 0.2222, Val: 0.3860, Test: 0.3660
torch.Size([509]) torch.Size([2708, 7])
Epoch: 005, Loss: 1.7888370752334595, Train: 0.2222, Val: 0.3940, Test: 0.3740
torch.Size([509]) torch.Size([2708, 7])
Epoch: 006, Loss: 1.7481361627578735, Train: 0.2222, Val: 0.4100, Test: 0.3840
torch.Size([509]) torch.Size([2708, 7])
Epoch: 007, Loss: 1.7075616121292114, Train: 0.2222, Val: 0.4280, Test: 0.3930
torch.Size([509]) torch.Size([2708, 7])
Epoch: 008, Loss: 1.6666404008865356, Train: 0.2222, Val: 0.4380, Test: 0.3960
torch.Size([509]) torch.Size([2708, 7])
Epoch: 00

In [9]:
# outs.shape, outs[0, 0, :].sum(), (outs[0, 0, :] == outs[2, 0, :]).sum()

In [10]:
best_val_acc = final_test_acc = 0

model.train()
for epoch in range(1, 100):
    optimizer.zero_grad()
    loss = model.loss()
    loss.backward()
    optimizer.step()
    
    train_acc, val_acc, tmp_test_acc = model.test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)

NameError: name 'optimizer' is not defined